In [25]:
import os


In [26]:
%pwd

'c:\\Users\\vkris\\Text-summarization'

In [4]:
os.chdir('../')

In [27]:
%pwd

'c:\\Users\\vkris\\Text-summarization'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokeniser_name: Path

In [29]:
from textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:

    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.DATA_PATH ,
            tokeniser_name = config.TOKENEISER_NAME
        )

        return data_transformation_config





In [31]:
from transformers import AutoTokenizer
from textsummarizer.logging import logger
from datasets import load_dataset,load_from_disk

In [32]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokeniser = AutoTokenizer.from_pretrained(self.config.tokeniser_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokeniser(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokeniser.as_target_tokenizer():
            target_encodings = self.tokeniser(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))    


In [33]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-11-26 10:48:49,681: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-26 10:48:49,694: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-26 10:48:49,697: INFO: common: created directory at: artifacts]
[2023-11-26 10:48:49,703: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 73910.83 examples/s]
